In [17]:
# ADMM adaptive lasso using ProximalOperators with line and golden section search
# Automatic tuning of learning rate and regularization parameter
# This code only only changes the ratio between test data and train data, and changes the input file
# The original source code from https://github.com/patwa67/AUTALASSO

using ProximalOperators
using DelimitedFiles
using Statistics
using LinearAlgebra

In [18]:
# Function for Golden section search to optimize lambda
function gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)
  lama =alam*maxnorm # Lower lambda
  lamb =blam*maxnorm # Higher lambda
  gr = (sqrt(5.0) + 1.0) / 2.0 # Golden section ratio
  toladmm = 1e-4 # Convergence tolerance for ADMM
  fc = lasso_admm(Xtrainhot, ytrain, lama, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
  fd = lasso_admm(Xtrainhot, ytrain, lamb, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
  iter = 2
  meanlam = zero(1.0:100.0)
  #meanlam[iter] = (lama+lamb)/2
  meanloss = zero(1.0:100.0)
  #meanloss[1] = max(lossc,lossd)
  #meanloss[iter] = (lossc+lossd)/2
  lamc = lamb - (lamb - lama) / gr
  lamd = lama + (lamb - lama) / gr
  println("lossc =$lossc")
  println("lossd =$lossd")
  println("lambdaa =$lama")
  println("lambdab =$lamb")
  println("lambdac =$lamc")
  println("lambdad =$lamd")
  iter = 2
  nodrun = 0
  while abs(lamc - lamd)/((lamc + lamd)/2.0) > tolgss # Run GSS until convergence
    iter = iter+1
    if iter == 3
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    else
    if nodrun==1
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    else
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    end
    end
    meanlam[iter] = (lamc+lamd)/2.0
    meanloss[iter] = (lossc+lossd)/2.0
    # Stop GSS if test MSE is increased two consecutive iterations
    if (meanloss[iter] > meanloss[iter-1])&&(meanloss[iter-1] > meanloss[iter-2])
      break
    end
    if lossc < lossd
      lamb = lamd
      fd=fc
      lossd=lossc
      nodrun=1
    else
      lama = lamc
      fc=fd
      lossc=lossd
      nodrun=0
    end
    lamc = lamb - (lamb - lama) / gr
    lamd = lama + (lamb - lama) / gr
    println("lambdac =$lamc")
    println("lambdad =$lamd")
  end
  # Final ADMM for optimized lambda
  lamopt = meanlam[iter-2]
  fmean1 = (fc[1]+fd[1])/2.0
  fmean2 = (fc[2]+fd[2])/2.0
  fopt = lasso_admm(Xtrainhot, ytrain, lamopt, fmean1,fmean2,f, abscovinv,toladmm)
  lossopt= 0.5*norm(Xtesthot*fopt[1].-ytest)^2
  acc = cor(Xtesthot*fopt[1],ytest)

  return lamopt,fopt,lossopt,acc
end

gss_opt (generic function with 1 method)

In [19]:
# Function for Proximal ADMM lasso with line search
function lasso_admm(Xtrainhot, ytrain, lam, theta, beta, f, abscovinv,tol; maxit=5000)
  u = zero(theta)
  grad = zero(theta)
  lamw = lam*abscovinv # Regularization parameter times weights
  g = NormL1(lamw) # Regularization function
  c = 0.5
  lr = 1.0
  loss(theta) = 0.5*norm(Xtrainhot*theta-ytrain)^2 # Loss function for line search
  for it = 1:maxit
    # Line search
    it % 8 == 1 && (grad = Xtrainhot'*(Xtrainhot*beta-ytrain))
    while  it % 20 == 2 && loss(theta) > (loss(beta) + grad'*(-beta) + (1.0/(2.0*lr))*norm(-beta)^2)
      lr = lr * c
      #println(lr)
    end
    gam = lr
    # ADMM perform f-update step
    prox!(beta, f, theta - u, gam)
    # ADMM perform g-update step
    prox!(theta, g, beta + u, gam)
    # Stopping criterion for ADMMM
    if norm(beta-theta, Inf) <= tol*(1+norm(u, Inf))
      break
    end
    # Dual update
    u .+= beta - theta
    #print(it)
  end
  return theta,beta,tol
end

lasso_admm (generic function with 1 method)

In [20]:
# The full path of the data file
data_file_path = "/Users/dong/GWAS/AtPolyDB/FT10.csv"

"/Users/dong/GWAS/AtPolyDB/FT10.csv"

In [21]:
X = readdlm(data_file_path,',')
ytot = (X[:,1].-mean(X[:,1])) # Center y to mean zero
ytrain = ytot[1:140]
Xtest= X[141:size(X)[1],2:size(X)[2]]
ytest = ytot[141:size(X)[1]]
Xtrain = X[1:140,2:size(X)[2]]

140×214051 Matrix{Float64}:
 0.0  0.0  0.0  2.0  2.0  0.0  0.0  2.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0  …  2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  2.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  2.0  2.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  2.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  

In [22]:
# One hot encoding training data
Xtrain0 = copy(Xtrain)
Xtrain1 = copy(Xtrain)
Xtrain2 = copy(Xtrain)
Xtrain0[Xtrain0.==1] .= 2
Xtrain0[Xtrain0.==0] .= 1
Xtrain0[Xtrain0.==2] .= 0
Xtrain1[Xtrain1.==2] .= 0
Xtrain2[Xtrain2.==1] .= 0
Xtrain2[Xtrain2.==2] .= 1
Xtrainhot = hcat(Xtrain0,Xtrain1,Xtrain2)
# Set unimportant allocations to zero
Xtrain0 = 0
Xtrain1 = 0
Xtrain2 = 0
Xtrain = 0

0

In [23]:
# One hot encoding test data
Xtest0 = copy(Xtest)
Xtest1 = copy(Xtest)
Xtest2 = copy(Xtest)
Xtest0[Xtest0.==1] .= 2
Xtest0[Xtest0.==0] .= 1
Xtest0[Xtest0.==2] .= 0
Xtest1[Xtest1.==2] .= 0
Xtest2[Xtest2.==1] .= 0
Xtest2[Xtest2.==2] .= 1
Xtesthot = hcat(Xtest0,Xtest1,Xtest2)
# Set unimportant allocations to zero
Xtest0 = 0
Xtest1 = 0
Xtest2 = 0
Xtest = 0

0

In [24]:
# Factor for initial lower lambda
alam = 0.0001
# Factor for initial upper lambda
blam = 1.0
# Convergence factor for lambda in gss_opt
tolgss = 0.01
# Find lambda where all reg coeff are zero
maxnorm = norm(Xtrainhot'*ytrain, Inf)
# The least squares loss function
f = LeastSquares(Xtrainhot, ytrain)
# Inverse covariances to be used as weights in the adaptive lasso
abscovinv = 1.0./abs.(cov(Xtrainhot,ytrain))

642153×1 Matrix{Float64}:
  0.4908749227490003
  0.4771888525152953
  0.406929937371266
  1.5005011951576834
  4.788975021533163
  3.159603831790878
  5.942892044586966
  0.7627334548376351
  0.4357074088171417
  1.343134209890603
  5.401804302567662
  0.4087977648467534
  2.843781966973549
  ⋮
  0.32814528775947244
  1.3653265979092122
 14.000000000000012
  0.7919744419347619
  3.4248504047870467
  0.8861162970720823
  0.37022592152199757
  0.8449297700974753
  1.5536926147704588
  0.3333076415829544
  1.072826506422625
  1.0706131543476467

In [25]:
# Run AUTALASSO with timing
@time res = gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)


lossc =4229.465063236628
lossd =5447.9315395512585
lambdaa =0.06713917525773198
lambdab =671.3917525773198
lambdac =256.49032401046225
lambdad =414.96856774211534
lambdac =158.5453829069108
lambdad =256.4903240104623
lambdac =98.0120802788093
lambdad =158.54538290691073
lambdac =60.6004418033592
lambdad =98.01208027880926
lambdac =98.01208027880926
lambdad =121.13374443146068
294.241630 seconds (83.96 k allocations: 39.915 GiB, 1.97% gc time, 0.05% compilation time)


(128.27873159286003, ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [2.9953105245561495e-6, -9.959809519621832e-6, 2.143747513783101e-6, -2.891102212687624e-6, -1.672410587198747e-5, -4.783041626582607e-5, -3.0690124234039606e-6, 2.3067393553427573e-5, 2.8729141469674424e-6, 1.5244717068846336e-6  …  -2.3209210238978734e-5, 2.4100559897877183e-5, 6.293371772502637e-6, -1.657054834069971e-6, 6.7641830112097345e-6, 2.014131795055607e-5, 3.865417310081054e-6, 8.292397134290752e-6, -8.27717802179273e-6, 1.3737009424308333e-5], 0.0001), 4268.057560980383, 0.7603619583818826)

In [26]:
output_dir = "/Users/dong/GWAS/Autalasso/"


"/Users/dong/GWAS/Autalasso/"

In [27]:
writedlm(joinpath(output_dir, "outbeta_FT10.txt"), res[2][1])
writedlm(joinpath(output_dir, "outlambda_FT10.txt"), res[1])
writedlm(joinpath(output_dir, "outMSE_FT10.txt"), res[3]/length(ytest)*2)
writedlm(joinpath(output_dir, "outACC_FT10.txt"), res[4])

In [28]:
# Predict observations for test data
ytesthat = Xtesthot*res[2][1]

54-element Vector{Float64}:
 -10.40480724642273
  -7.752280838005121
  -8.815209609588186
   0.827694839998522
  33.960274148798455
   1.8881055014617831
 -14.101722812773207
  -1.6821627107787962
   6.204319259891909
   6.833472980701151
  -6.397185037622437
  17.644217418000494
  -3.898030489363935
   ⋮
  27.105575515252884
   4.213089968097407
  -6.68884363611132
   5.032919630809717
  -8.161652404992692
   4.777885759399673
  -8.91970630495024
 -17.307410475098273
   0.45648517547991013
  -2.069493952427322
 -15.219667246180501
  -8.391363577616787